# A Data Cleaning Problem That I Encountered And The Solution That I Built To Fix It

I will be using two datasets to demonstrate this problem. 
The titanic and diabetes dataset.

The problem is this: After i built a pipeline using SimpleImputer, OneHotEncoder, ColumnTransformer. The dataset still had missing values.

I found a solution to this, but this solution doesn't seem as though it could scale when given a dataset without manual listsing of the dataset features. 

Let me start the demonstration with the Titanic dataset

In [25]:
import pandas as pd
import numpy as np

In [26]:
#titanic dataset
df2 = pd.read_csv('titanic.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [27]:
#check for missing values
df2.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
#identifying the numerical features
num_cols = df2.select_dtypes(include=['number']).columns
num_cols

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [6]:
#identifying the categorical features
cate_cols = df2.select_dtypes(include=['object']).columns
cate_cols

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [7]:
cat_feature = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
num_feature = ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [9]:
from sklearn.impute import SimpleImputer

In [10]:
# grouping these features and building a pipeline with it
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
num_feature = ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

cat_feature = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder())
])

prep = ColumnTransformer([
    ("numerical", num_pipeline, num_cols),
    ("categorical", cat_pipeline, cate_cols)
])

In [13]:
prep

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object'))])

In [15]:
prep.fit_transform(df2).toarray()

array([[  1.,   0.,   3., ...,   0.,   0.,   1.],
       [  2.,   1.,   1., ...,   1.,   0.,   0.],
       [  3.,   1.,   3., ...,   0.,   0.,   1.],
       ...,
       [889.,   0.,   3., ...,   0.,   0.,   1.],
       [890.,   1.,   1., ...,   1.,   0.,   0.],
       [891.,   0.,   3., ...,   0.,   1.,   0.]])

The dataframe still has missing values

In [16]:
df2.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Observation: Age, Cabin and Embarked columns has missing values

I want to try these following codes

In [17]:
numeric_imputer = SimpleImputer(strategy='median')

df2[num_feature] = numeric_imputer.fit_transform(df2[num_feature])
df_num = df2[num_feature]
df_num.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500


In [18]:
#check for missing values again
df2.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

It can be observed that only the numeric features of this dataset missing values were cleaned by the previous code.
What now happens to the categorical features?

In [19]:
cat_imput = SimpleImputer(strategy="most_frequent")

df2[cat_feature] = cat_imput.fit_transform(df2[cat_feature])

df_cat = df2[cat_feature]
df_cat.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,B96 B98,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,B96 B98,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,B96 B98,S


In [20]:
df2.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

If you notice, this dataset now has no missing values

# How About Cleaning Everything at Once?

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np


# Identify numeric and categorical features
num_feature = df2.select_dtypes(include=["number"]).columns.tolist()
cat_feature = df2.select_dtypes(include=["object"]).columns.tolist()

# Numeric pipeline
numeric_imputer = SimpleImputer(strategy='median')
num_pipeline = Pipeline([
    ("imputer", numeric_imputer),
    ("standardize", StandardScaler())
])

# Apply numeric imputation to the DataFrame
df2[num_feature] = numeric_imputer.fit_transform(df2[num_feature])
df_num = df2[num_feature]

# Categorical pipeline
cat_imputer = SimpleImputer(strategy="most_frequent")
cat_pipeline = Pipeline([
    ("imputer", cat_imputer),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Apply categorical imputation to the DataFrame
df2[cat_feature] = cat_imputer.fit_transform(df2[cat_feature])
df_cat = df2[cat_feature]

# Combine the pipelines into a single ColumnTransformer
prep = ColumnTransformer([
    ("numerical", num_pipeline, num_feature),
    ("categorical", cat_pipeline, cat_feature)
])

# Apply the pipeline to the dataset
cleaned_data = prep.fit_transform(df2)
cleaned_data

<891x1731 sparse matrix of type '<class 'numpy.float64'>'
	with 10691 stored elements in Compressed Sparse Row format>

In [31]:
prep

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardize',
                                                  StandardScaler())]),
                                 ['PassengerId', 'Survived', 'Pclass', 'Age',
                                  'SibSp', 'Parch', 'Fare']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Name', 'Sex', 'Ticket', 'Cabin',
                                  'Embarked'])])

In [29]:
df2.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [32]:
df2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,B96 B98,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,B96 B98,S
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5.0,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887.0,0.0,2.0,"Montvila, Rev. Juozas",male,27.0,0.0,0.0,211536,13.0000,B96 B98,S
887,888.0,1.0,1.0,"Graham, Miss. Margaret Edith",female,19.0,0.0,0.0,112053,30.0000,B42,S
888,889.0,0.0,3.0,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1.0,2.0,W./C. 6607,23.4500,B96 B98,S
889,890.0,1.0,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.0000,C148,C


In [34]:
cleaned_data = prep.fit_transform(df2).toarray()
cleaned_data

array([[-1.73010796, -0.78927234,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [-1.72622007,  1.2669898 , -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [-1.72233219,  1.2669898 ,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.72233219, -0.78927234,  0.82737724, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.72622007,  1.2669898 , -1.56610693, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.73010796, -0.78927234,  0.82737724, ...,  0.        ,
         1.        ,  0.        ]])